### **Multi-Layer Perceptron (MLP) Neural Network**

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
from sklearn.metrics import root_mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm.notebook import trange

In [2]:
# LOAD PROCESSED DATA
df = pd.read_csv("processed_drug_consumption.csv")
# Specify input & target columns
target_cols = [
    'dissociatives', 'cannabinoids', 'empathogens',
    'depressants_pca', 'opioids_pca', 'stimulants_pca', 'psychedelics_pca', 'inhalants_pca'
]
input_cols = [col for col in df.columns if col not in target_cols] # there're FAR more input columns after one-hot encoding nominal variables

X = df[input_cols].values.astype('float32')
y = df[target_cols].values.astype('float32')

#### **Prepare Data for PyTorch**

In [3]:
# TRANSFORM DATA FOR PYTORCH

# Convert numpy representation of dataframes into compatible, multi-dimensional arrays (which can run on either CPU or GPU!)
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)

In [4]:
# CREATE PYTORCH DATALOADER

# Make PyTorch TensorDataset to be passed to the DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
# Specify train/test dataset sizes (80/20 split), and split the dataset into two aforementioned datasets
train_size = int(0.80 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, lengths=[train_size, test_size])

# Create PyTorch DataLoaders to "simplify loading and iterating over datasets while training deep learning models" https://www.geeksforgeeks.org/deep-learning/pytorch-dataloader/ 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # Add shuffling to training data to prevent model from "memorizing" data order and finding unreliable/dubious patterns (not needed when testing)
test_loader = DataLoader(test_dataset, batch_size=32) # 32 is common batch size for smaller datasets

#### **Build/Configure Model**

In [5]:
# BUILD MLP MODEL

class MLP(nn.Module):
    # Model initialization with number of input/output features and (opt.) number of neurons in the hidden layers
    def __init__(self, input_size, output_size, hidden_neurons=64):
        super().__init__() # initializes parent class
        # Make the fully-connected/dense layers (having only one layer would make a linear model)
        self.fc1 = nn.Linear(input_size, hidden_neurons) # Input
        self.fc2 = nn.Linear(hidden_neurons, hidden_neurons)
        self.fc3 = nn.Linear(hidden_neurons, output_size) # Output
    # Setup forward passing (data flow through the network)
    def forward(self, x):
        # Pass through the layers and apply ReLU activation (popular activation function, also add "non-linearity") after hidden layers (where app.) 
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

model = MLP(input_size=X.shape[1], output_size=y.shape[1])

In [6]:
# SETUP LOSS FUNCTION & OPTIMIZER

# Setup optimizer (Adam is commonly used, it adapts learning rate per parameter from model.parameters())
optimizer = torch.optim.Adam(model.parameters())
# Use Mean Squared Error for the loss function (default for most regression problems)
criterion = nn.MSELoss()

#### **Train & Evalute the Model**

In [7]:
# TRAIN MODEL

# Loop through epochs, training the model X times with the training data (higher X, more learning -> too high X, overfitting)
epochs = 50
progress_bar = trange(epochs, desc="Training", dynamic_ncols=True)
for epoch in progress_bar:
    # Sets model in training mode and create accumulator for epoch's total loss 
    model.train()
    running_loss = 0.0
    total_samples = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad() # Reset gradient values (prevent unwanted accumulation)
        predictions = model(batch_X) # Forward pass to get predictions
        # Calculate loss and backpropogate (calculate loss gradients w.r.t. model parameters)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step() # Update weights w/ Adam (optimizer algorithm)
        batch_size = batch_X.size(0)
        running_loss += loss.item() * batch_X.size(0)
        total_samples += batch_size
    avg_loss = running_loss / total_samples
    progress_bar.set_postfix(epoch=epoch+1, avg_loss=f"{avg_loss:.4f}")

Training:   0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
# EVALUATE MODEL

# Sets model in evalation mode
model.eval()
with torch.no_grad():   # turn off gradient computation, unnecessary and saves memory
    # Prepare lists to store each batch's predictions and the true values 
    all_preds = []
    all_targets = []
    for batch_X, batch_y in test_loader: # iterates and appends through test data
        preds = model(batch_X)
        all_preds.append(preds.numpy())
        all_targets.append(batch_y.numpy())
    all_preds = np.vstack(all_preds)
    all_targets = np.vstack(all_targets)
# Calculate overall root mean squared error (standard regression metric) 
rmse = root_mean_squared_error(all_targets, all_preds)
print("Test RMSE (overall):", rmse)

Test RMSE (overall): 0.9835370779037476


#### **Save Model Weights**

In [9]:
torch.save(model.state_dict(), "mlp_drug_predictor.pth")
print("Model weights saved!")

Model weights saved!
